In [3]:
!pip install scikit-image

  Using cached scikit_image-0.23.2-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached imageio-2.34.1-py3-none-any.whl.metadata (4.9 kB)
  Using cached tifffile-2024.4.24-py3-none-any.whl.metadata (31 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached scikit_image-0.23.2-cp310-cp310-win_amd64.whl (12.7 MB)
Using cached imageio-2.34.1-py3-none-any.whl (313 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
Using cached tifffile-2024.4.24-py3-none-any.whl (225 kB)


In [6]:
!pip install filterpy

  Using cached filterpy-1.4.5-py3-none-any.whl


In [7]:
!pip list

Package            Version
------------------ --------------------
asttokens          2.4.1
certifi            2024.2.2
charset-normalizer 3.3.2
colorama           0.4.6
comm               0.2.2
contourpy          1.2.1
cvzone             1.6.1
cycler             0.12.1
debugpy            1.8.1
decorator          5.1.1
exceptiongroup     1.2.1
executing          2.0.1
filelock           3.14.0
filterpy           1.4.5
fonttools          4.51.0
fsspec             2024.3.1
idna               3.7
imageio            2.34.1
intel-openmp       2021.4.0
ipykernel          6.29.4
ipython            8.24.0
jedi               0.19.1
Jinja2             3.1.3
jupyter_client     8.6.1
jupyter_core       5.7.2
kiwisolver         1.4.5
lazy_loader        0.4
MarkupSafe         2.1.5
matplotlib         3.8.4
matplotlib-inline  0.1.7
mkl                2021.4.0
mpmath             1.3.0
nest-asyncio       1.6.0
networkx           3.3
numpy              1.26.4
opencv-python      4.9.0.80
packaging       

In [8]:
from ultralytics import YOLO
from sort import *
import cv2
import cvzone
import math

In [10]:
cap = cv2.VideoCapture("../Videos/cars.mp4")

In [11]:
model = YOLO("../YOLO-Weights/yolov8l.pt")

classNames = [
    "person",
    "bicycle",
    "car",
    "motorbike",
    "aeroplane",
    "bus",
    "train",
    "truck",
    "boat",
    "traffic light",
    "fire hydrant",
    "stop sign",
    "parking meter",
    "bench",
    "bird",
    "cat",
    "dog",
    "horse",
    "sheep",
    "cow",
    "elephant",
    "bear",
    "zebra",
    "giraffe",
    "backpack",
    "umbrella",
    "handbag",
    "tie",
    "suitcase",
    "frisbee",
    "skis",
    "snowboard",
    "sports ball",
    "kite",
    "baseball bat",
    "baseball glove",
    "skateboard",
    "surfboard",
    "tennis racket",
    "bottle",
    "wine glass",
    "cup",
    "fork",
    "knife",
    "spoon",
    "bowl",
    "banana",
    "apple",
    "sandwich",
    "orange",
    "broccoli",
    "carrot",
    "hot dog",
    "pizza",
    "donut",
    "cake",
    "chair",
    "sofa",
    "pottedplant",
    "bed",
    "diningtable",
    "toilet",
    "tvmonitor",
    "laptop",
    "mouse",
    "remote",
    "keyboard",
    "cell phone",
    "microwave",
    "oven",
    "toaster",
    "sink",
    "refrigerator",
    "book",
    "clock",
    "vase",
    "scissors",
    "teddy bear",
    "hair drier",
    "toothbrush",
]

In [12]:
mask = cv2.imread("mask.png")

In [13]:
# Tracking
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

limits = [400, 297, 673, 297]
totalCount = []

In [14]:
while True:
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)

    imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (0, 0))

    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100

            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if (
                currentClass == "car"
                or currentClass == "motorbike"
                or currentClass == "bus"
                or currentClass == "truck"
                and conf > 0.3
            ):
                # cvzone.putTextRect(
                #     img,
                #     f"{currentClass} {conf}",
                #     (max(0, x1), max(35, y1)),
                #     scale=0.8,
                #     thickness=1,
                #     offset=3,
                # )
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    resultsTracker = tracker.update(detections)

    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1
        cvzone.putTextRect(
            img,
            f"{int(id)}",
            (max(0, x1), max(35, y1)),
            scale=2,
            thickness=3,
            offset=10,
        )

        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(
                    img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5
                )

    # cvzone.putTextRect(img, f"Count: {len(totalCount)}", (50, 50))
    cv2.putText(
        img,
        str(len(totalCount)),
        (255, 100),
        cv2.FONT_HERSHEY_PLAIN,
        5,
        (50, 50, 255),
        8,
    )

    cv2.imshow("Image", img)
    # cv2.imshow("ImageRegion", imgRegion)
    cv2.waitKey(1)


0: 384x640 3 cars, 163.1ms
Speed: 3.3ms preprocess, 163.1ms inference, 696.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 16.6ms
Speed: 1.0ms preprocess, 16.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 17.9ms
Speed: 1.0ms preprocess, 17.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 18.0ms
Speed: 1.0ms preprocess, 18.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 24.8ms
Speed: 2.0ms preprocess, 24.8ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 24.2ms
Speed: 1.3ms preprocess, 24.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 17.5ms
Speed: 1.4ms preprocess, 17.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 23.2ms
Speed: 1.5ms preprocess, 23.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'
